# Deliverable 2. Create a Customer Travel Destinations Map

----

1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Geoapify API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found".

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame. In the point for each city add:

    * The city name
    
    * The country code
    
    * The weather description
    
    * The point's size should be the maximum temperature for the city

15. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.

---

## Ensure that the dependencies and the Geoapify API key is imported correctly

In [24]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`

In [25]:
# Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv('../Weather_Database/weatherpy_database.csv')

city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Alofi,NU,-19.0595,-169.9187,78.69,83,52,5.75,broken clouds
1,1,Ushuaia,AR,-54.8000,-68.3000,53.26,58,75,5.75,broken clouds
2,2,Richards Bay,ZA,-28.7830,32.0377,70.97,95,4,5.17,clear sky
3,3,Vaini,TO,-21.2000,-175.2000,80.76,94,75,8.05,light intensity drizzle
4,4,Kaeo,NZ,-35.1000,173.7833,72.32,70,24,13.73,few clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation

In [26]:
# Prompt the user to enter minimum and maximum temperature criteria.
min_temp = float(input('What is the minimum temperature you would like for your trip?'))
max_temp = float(input('What is the maximum temperature you would like for your trip?'))

## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`

In [27]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & (city_data_df['Max Temp'] >= min_temp)]

# Display sample data
preferred_cities_df.head()


,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Alofi,NU,-19.0595,-169.9187,78.69,83,52,5.75,broken clouds
2,2,Richards Bay,ZA,-28.7830,32.0377,70.97,95,4,5.17,clear sky
3,3,Vaini,TO,-21.2000,-175.2000,80.76,94,75,8.05,light intensity drizzle
4,4,Kaeo,NZ,-35.1000,173.7833,72.32,70,24,13.73,few clouds
5,5,Avarua,CK,-21.2078,-159.7750,80.65,69,75,6.91,broken clouds


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows

In [28]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_travel_cities = city_data_df.dropna()

# Display sample data
clean_travel_cities.isnull().sum()

Unnamed: 0             0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Weather Description    0
dtype: int64

## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng"

In [29]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_travel_cities[['City', 'Country', 'Max Temp', 'Lat', 'Lng', 'Weather Description']].copy()

## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame

In [30]:
# Create a new column "Hotel Name".
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Weather Description,Hotel Name
0,Alofi,NU,78.69,-19.0595,-169.9187,broken clouds,
1,Ushuaia,AR,53.26,-54.8000,-68.3000,broken clouds,
2,Richards Bay,ZA,70.97,-28.7830,32.0377,clear sky,
3,Vaini,TO,80.76,-21.2000,-175.2000,light intensity drizzle,
4,Kaeo,NZ,72.32,-35.1000,173.7833,few clouds,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city

In [31]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

## Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found"

In [32]:
# Iterate through the hotel DataFrame

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame.
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set up the base URL for the Geoapify Places API.
    base_url = "https://api.geoapify.com/v2/places"

    # Make request and retrieve the JSON data by using the params dictionaty
    name_address = requests.get(base_url, params= params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found set the hotel name as "No hotel found".
    try:
        hotel_df.loc[index, 'Hotel Name'] = name_address['features'][0]['properties']['name']
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Alofi - nearest hotel: No hotel found
Ushuaia - nearest hotel: Apart Hotel Aires del Beagle
Richards Bay - nearest hotel: Protea Hotel
Vaini - nearest hotel: Keleti International Resort
Kaeo - nearest hotel: No hotel found
Avarua - nearest hotel: Paradise Inn
Saint-Augustin - nearest hotel: No hotel found
Arraial Do Cabo - nearest hotel: No hotel found
San Patricio - nearest hotel: No hotel found
Peniche - nearest hotel: Residencial Maciel
Barrow - nearest hotel: No hotel found
Busselton - nearest hotel: No hotel found
Lata - nearest hotel: No hotel found
Klaksvik - nearest hotel: Hotel Klaksvik
Paamiut - nearest hotel: Hotel Ivaana
Saint George - nearest hotel: The Advenire, an Autograph Collection Hotel
Esperance - nearest hotel: Hospitality
La Ronge - nearest hotel: La Ronge Hotel and Suites
Bengkulu - nearest hotel: Dena Hotel
Iqaluit - nearest hotel: Frobisher Inn
Tasiilaq - nearest hotel: Angmagssalik
Rikitea - nearest hotel: Chez Bianca & Benoit
Kintampo - 

,City,Country,Max Temp,Lat,Lng,Weather Description,Hotel Name
0,Alofi,NU,78.69,-19.0595,-169.9187,broken clouds,No hotel found
1,Ushuaia,AR,53.26,-54.8000,-68.3000,broken clouds,Apart Hotel Aires del Beagle
2,Richards Bay,ZA,70.97,-28.7830,32.0377,clear sky,Protea Hotel
3,Vaini,TO,80.76,-21.2000,-175.2000,light intensity drizzle,Keleti International Resort
4,Kaeo,NZ,72.32,-35.1000,173.7833,few clouds,No hotel found
...,...,...,...,...,...,...,...
701,Udachnyy,RU,-60.48,66.4167,112.4000,broken clouds,Вилюй
702,Kedrovyy,RU,10.87,56.1667,91.8167,overcast clouds,No hotel found
703,Ndioum,SN,74.95,16.5129,-14.6471,clear sky,No hotel found
704,Puerto Baquerizo Moreno,EC,74.32,-0.9000,-89.6000,few clouds,Dory's House


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`

In [33]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df['Hotel Name'] != 'No hotel found']

# Display sample data
clean_hotel_df

,City,Country,Max Temp,Lat,Lng,Weather Description,Hotel Name
1,Ushuaia,AR,53.26,-54.8000,-68.3000,broken clouds,Apart Hotel Aires del Beagle
2,Richards Bay,ZA,70.97,-28.7830,32.0377,clear sky,Protea Hotel
3,Vaini,TO,80.76,-21.2000,-175.2000,light intensity drizzle,Keleti International Resort
5,Avarua,CK,80.65,-21.2078,-159.7750,broken clouds,Paradise Inn
9,Peniche,PT,56.10,39.3558,-9.3811,overcast clouds,Residencial Maciel
...,...,...,...,...,...,...,...
698,Byron Bay,AU,81.95,-28.6500,153.6167,broken clouds,Lord Byron Resort
699,Sadao,TH,76.82,6.6388,100.4234,broken clouds,S.T. Hotel
701,Udachnyy,RU,-60.48,66.4167,112.4000,broken clouds,Вилюй
704,Puerto Baquerizo Moreno,EC,74.32,-0.9000,-89.6000,few clouds,Dory's House


## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the `Vacation_Search`` folder

In [34]:
# Create the output File (CSV)
output_file = 'weatherpy_vacation.csv'

# Export the City_Data into a csv
clean_hotel_df.to_csv(output_file, index_label = 'city_id')

## Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame.

In the point for each city add:

* The city name
* The country code
* The weather description
* The point's size should be the maximum temperature for the city

In [35]:
# Configure the map plot
map_plot_1 = clean_hotel_df.hvplot.points(
    'Lng', 
    'Lat', 
    geo = True, 
    tiles = 'OSM', 
    size = 'Max Temp', 
    scale = 1, 
    color = 'City', 
    hover_cols = ['City', 'Country', 'Weather Description']
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Country,Weather Description)